#2

In [1]:
import pyspark as psk
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyinputplus as pyin
import warnings
import mysql.connector 


In [2]:
import sys
import os
import pandas.io.sql as psql
import findspark
findspark.init()

In [3]:
# suppress warnings
warnings.filterwarnings('ignore')

In [4]:
# create a SparkSession
spark = SparkSession.builder \
    .appName("Bank Transactions") \
    .config("spark.jars","/Users/roy/Downloads/mysql-connector-j-8.0.32/mysql-connector-j-8.0.32.jar")\
    .getOrCreate()


23/03/01 23:22:32 WARN Utils: Your hostname, Rajarshis-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.159 instead (on interface en0)
23/03/01 23:22:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/01 23:22:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
query = "(SELECT cc.*, cust.cust_zip \
        FROM cdw_sapp_credit_card as cc \
        JOIN cdw_sapp_customer as cust ON cc.CUST_SSN = cust.SSN) as i"


# define MySQL connection parameters
df = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", query) \
    .option("user", "root") \
    .option("password", "ShaShi3493*") \
    .load()

df = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", query) \
    .option("user", "root") \
    .option("password", "ShaShi3493*") \
    .load()

Py4JJavaError: An error occurred while calling o33.load.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:107)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:107)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:39)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:34)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:350)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:228)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:210)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.

In [ ]:
query = "(SELECT * FROM cdw_sapp_branch bc \
      JOIN cdw_sapp_credit_card cc ON bc.BRANCH_CODE = cc.BRANCH_CODE \
        JOIN cdw_sapp_customer as cust ON cc.CUST_SSN = cust.SSN)"

# define a function to execute SQL queries and return results
def execute_query(query):
    return spark.sql(query).toPandas()
print(execute_query(query))

In [ ]:


query = "(SELECT cc.*, cust.cust_zip \
        FROM cdw_sapp_credit_card as cc \
        JOIN cdw_sapp_customer as cust ON cc.CUST_SSN = cust.SSN) as a"
df_sp_cc_cust = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", query) \
    .option("user", "root") \
    .option("password", "ShaShi3493*") \
    .load()

print(df)         

In [ ]:
df.show(1)

In [ ]:
    # convert the Spark DataFrame to a Pandas DataFrame
pd_df = df.toPandas()

# display the first 5 rows of the Pandas DataFrame
print(pd_df.head(5))

In [ ]:


# define a function to execute SQL queries and return results
def execute_query(query, mysql_hostname, mysql_port, mysql_database, mysql_username, mysql_password):
    url = f"jdbc:mysql://{mysql_hostname}:{mysql_port}/{mysql_database}"
    properties = {
        "driver": "com.mysql.jdbc.Driver",
        "user": mysql_username,
        "password": mysql_password
    }
    df = spark.read.jdbc(url=url, table=f"({query}) as t", properties=properties)
    return df.toPandas()


In [ ]:
df.head(2)


# establish database connection
cnx = mysql.connector.connect(user='root', password='ShaShi3493*',
                              host='localhost', database='creditcard_capstone')

# execute SQL query and store results in dataframe
df = pd.read_sql("(SELECT * FROM cdw_sapp_branch bc JOIN cdw_sapp_credit_card cc ON bc.BRANCH_CODE = cc.BRANCH_CODE JOIN cdw_sapp_customer as cust ON cc.CUST_SSN = cust.SSN)", con=cnx)

df.head(2)

In [ ]:
# define a function to execute SQL queries and return a PySpark DataFrame

query = "(SELECT * FROM cdw_sapp_branch bc JOIN cdw_sapp_credit_card cc ON bc.BRANCH_CODE = cc.BRANCH_CODE \
        JOIN cdw_sapp_customer as cust ON cc.CUST_SSN = cust.SSN)"


def execute_query(query):
    df = spark.read.format("jdbc")\
        .option("url", f"jdbc:mysql://{mysql_hostname}:{mysql_port}/{mysql_database}")\
            .option("driver", "com.mysql.jdbc.Driver")\
                .option("user", mysql_username)\
                    .option("password", mysql_password)\
                        .option("dbtable", "({})".format(query))\
                            .load()
    return df           
    # convert the Spark DataFrame to a Pandas DataFrame
    pd_df = df.toPandas()

# display the first 5 rows of the Pandas DataFrame
print(pd_df.head(5))

In [ ]:
#1)    Used to display the transactions made by customers living in a given zip code for a given month and year. 
def execute_query():

    flag = True
    zip_code = input("Enter Your Zip Code:  ")
    while (flag):
        if not zip_code.isdigit():
            zip_code = input('enter a valid zipcode')
        else:
            flag=False
    
    transaction_year = input("Enter the year for transactions records:  ")
    flag=True
    while (flag):
        if not transaction_year.isdigit() or len(transaction_year)!=4:
            transaction_year = input("Enter valid year format XXXX")
        else:
            flag = False
        
    transaction_month = input("Enter the month in the form (XX): ").zfill(2)
    flag = True
    while (flag):
        if  not transaction_month.isdigit():
            transaction_month = input('Please enter valid month')
        else:
            flag=False
    

    cursor.execute(query,(zip_code,transaction_year,transaction_month,))
    records = cursor.fetchall()
    print("\n List of  customer transactions in a given Zip code : ")
    print("\n\n Customer no\t\tdate\t   cust_ssn  code  type       value  id")
    for row in records:
        print(row)


In [ ]:
# define a function to validate user input

def validate_input(prompt, expected_type):
    while True:
        user_input = pyin.inputStr(prompt)
        try:
            validated_input = expected_type(user_input)
            return validated_input
        except ValueError:
            print("Invalid input. Please enter a {}.".format(expected_type.__name__))

In [ ]:
while True:
    print("Select an option:")
    print("1. Display transactions by zip code and month/year")
    print("2. Display number and total value of transactions by type")
    print("3. Display total number and value of transactions by state")
    print("4. Exit")

    # validate user input
    choice = validate_input("Enter your choice: ", int)
    if choice == 1:
        #1)    Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.
       
        zip_code = validate_input("Enter the zip code: ", int)
        year = validate_input("Enter the year (YYYY): ", int)
        month = validate_input("Enter the month (MM): ", int)
        result = execute_query("SELECT * FROM CDW_SAPP_CUSTOMER, CDW_SAPP_BRANCH WHERE CUST_ZIP = {} AND YEAR(date) = {} AND MONTH(date) = {} ORDER BY date DESC".format(zip_code, year, month))
        if result:
            pd_result = result.toPandas()
            print(pd_result)
        else:
            print("No transactions found.")
    elif choice == 2:
        #2)    Used to display the number and total values of transactions for a given type.
      
        transaction_type = validate_input("Enter the transaction type: ", str)
        result = execute_query("SELECT TRANSACTION_VALUE, TRANSACTION_TYPE FROM '{}' where TRANSACTION_TYPE  = '{}'".format(query, transaction_type))
        if result:
            pd_result = result.toPandas()
            print(pd_result)
        else:
            print("No transactions found.")
    elif choice == 3:
        #3)    Used to display the total number and total values of transactions for branches in a given state.
    
        state = validate_input("Enter the state: ", str)
        result = execute_query("SELECT bc.state, COUNT(*) as count, SUM(TRANSACTION_VALUE) as total_value FROM cdw_sapp_branch bc JOIN cdw_sapp_credit_card cc ON bc.BRANCH_CODE = cc.BRANCH_CODE WHERE BRANCH_STATE = '{}'".format(state))
        if result:
            pd_result = result.toPandas()
            print(pd_result)
        else:
            print("No transactions found.")
    elif choice == 4:
        # exit
        break
    else:
        print("Invalid choice.")


In [ ]:
#stop Spark
spark.stop()